<a href="https://colab.research.google.com/github/bodadineshreddy/indictrans2/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# !rm -rf /content/

In [1]:
%%capture
# !git clone https://github.com/AI4Bharat/IndicTrans2.git
# %cd /content/IndicTransToolkit
# !git clone https://github.com/VarunGumma/IndicTransToolkit.git
!pip install git+https://github.com/VarunGumma/IndicTransToolkit.git
# !python3 -m pip install --editable ./
!python3 -c "import nltk; nltk.download('punkt')"

!pip install transformers datasets torch sentencepiece sacrebleu bitsandbytes scipy accelerate
!pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [3]:
import os
import json
from datasets import load_dataset, Dataset
from itertools import islice

# Define source and target languages (must match both dataset & model)
src_lang, tgt_lang = "eng_Latn", "tel_Telu"

# Load dataset specifying the language pair (STREAMING mode enabled)
print(f"Loading dataset: {src_lang} → {tgt_lang}")
dataset_train = load_dataset("allenai/nllb", f"{src_lang}-{tgt_lang}", split="train", streaming=True, trust_remote_code=True)

# Reduce dataset size (e.g., 2000 samples instead of full dataset)
SAMPLE_SIZE = 2000  # Adjust as needed
batch_samples = list(islice(dataset_train, SAMPLE_SIZE))  # Efficiently fetch only needed samples

# Convert lists into Hugging Face Datasets
dataset = Dataset.from_list(batch_samples)['translation']

print(f"Loaded {len(dataset)}, {dataset[:2]}")

# Convert dataset to list format
en_to_te = [{"src": ex[src_lang], "tgt": ex[tgt_lang]} for ex in dataset]

# Generate bilingual pairs
te_to_en = [{"src": ex[tgt_lang], "tgt": ex[src_lang]} for ex in dataset]

# Convert generators to lists
json_data = {
    "en-indic": list(en_to_te),
    "indic-en": list(te_to_en)
}

# Save to a single JSON file
with open("nllb_en_te.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)

print("Dataset saved as nllb_en_te.json")

Loading dataset: eng_Latn → tel_Telu


Repo card metadata block was not found. Setting CardData to empty.


Loaded 2000, [{'eng_Latn': 'I fear for you a day on which will be a great outcry!"', 'tel_Telu': 'వాస్తవానికి నేను మీపై రాబోయే ఆ గొప్ప దినపు శిక్షను గురించి భయపడుతున్నాను\'అని అన్నాడు" (7:59)'}, {'eng_Latn': 'Which is indeed a great oath if only you knew it.', 'tel_Telu': 'మీరు గ్రహించగలిగితే ఇది గొప్ప ప్రమాణం.'}]
Dataset saved as nllb_en_te.json


In [8]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset

# Load Pre-trained Model and Tokenizer
model_name = "ai4bharat/indictrans2-indic-en-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Set padding token explicitly
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, trust_remote_code=True)
model.config.pad_token_id = tokenizer.pad_token_id

# Load Dataset
dataset = load_dataset("json", data_files="nllb_en_te.json", split="train")

# Convert Dataset to Required Format
dataset = Dataset.from_list(dataset["indic-en"])

# Display a sample to verify structure
print("Sample Data:", dataset[0])

# Split dataset into train and validation sets
split_dataset = dataset.train_test_split(test_size=0.1, seed=42)

train_dataset = split_dataset["train"]
valid_dataset = split_dataset["test"]

# Function to Tokenize Data with Fixes
def tokenize(batch):
    source_texts = batch["src"]
    target_texts = batch["tgt"]

    # Tokenize inputs without forcing max_length
    source_encodings = tokenizer(source_texts, padding="longest", truncation=True, max_length=128)

    # Tokenize targets without forcing max_length
    target_encodings = tokenizer(target_texts, padding="longest", truncation=True, max_length=128)
    labels = target_encodings["input_ids"]

    # Ensure `labels` is always a list of lists
    if isinstance(labels[0], int):
        labels = [labels]

    # Replace **only padding tokens** in labels with -100
    labels = [
        [token if token != tokenizer.pad_token_id else -100 for token in sequence]
        for sequence in labels
    ]

    return {
        "input_ids": source_encodings["input_ids"],
        "attention_mask": source_encodings["attention_mask"],
        "labels": labels
    }


# Apply tokenization separately to train and validation datasets
tokenized_train_dataset = train_dataset.map(tokenize, batched=True, remove_columns=train_dataset.column_names)
tokenized_valid_dataset = valid_dataset.map(tokenize, batched=True, remove_columns=valid_dataset.column_names)

# Remove empty or corrupt samples
tokenized_train_dataset = tokenized_train_dataset.filter(lambda x: len(x["input_ids"]) > 0 and len(x["labels"]) > 0)
tokenized_valid_dataset = tokenized_valid_dataset.filter(lambda x: len(x["input_ids"]) > 0 and len(x["labels"]) > 0)

# Set Training Parameters
training_args = TrainingArguments(
    output_dir="fine_tuned_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=0.00003,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    logging_dir="logs",
    logging_steps=50
)

# Initialize Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    data_collator=data_collator
)

# Train Model
trainer.train()

# Save Model and Tokenizer
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")

# Test Translation
input_text = "मुझे स्कूल जाना है।"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Ensure model uses the correct pad token during inference
output_ids = model.generate(**inputs, pad_token_id=tokenizer.pad_token_id)

output_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]

# Print Translated Output
print("Translated Output:", output_text)


Sample Data: {'src': 'వాస్తవానికి నేను మీపై రాబోయే ఆ గొప్ప దినపు శిక్షను గురించి భయపడుతున్నాను\'అని అన్నాడు" (7:59)', 'tgt': 'I fear for you a day on which will be a great outcry!"'}


Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1800 [00:00<?, ? examples/s]

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


IndexError: index out of range in self

In [2]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
from IndicTransToolkit import IndicProcessor

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert", trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )
    # Resize model embedding to match the new tokenizer's vocab size
    new_vocab_size = len(tokenizer)
    model.resize_token_embeddings(new_vocab_size)

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.bfloat16()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

# ==============================
# Dataset Loading (Using allenai/nllb)
# ==============================

# Define source and target languages (must match both dataset & model)
src_lang, tgt_lang = "eng_Latn", "tel_Telu"

# Load dataset specifying the language pair (STREAMING mode enabled)
dataset = load_dataset("allenai/nllb", f"{src_lang}-{tgt_lang}", split="train", streaming=True, trust_remote_code=True)

# Collect 10,000 samples into a list (since streaming datasets do not support .map() directly)
batch_samples = [x for _, x in zip(range(10000), dataset)]

# Convert the list into a Hugging Face Dataset
dataset = Dataset.from_list(batch_samples)

print(dataset[0])

# ==============================
# Model Initialization (Uses Existing Functions)
# ==============================

# Load the pretrained English-to-Indic model
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)

# Create an instance of IndicProcessor
ip = IndicProcessor(inference=True)

# ==============================
# Preprocessing for Fine-Tuning
# ==============================

# Define the preprocessing function (handling single dictionary input)
def preprocess_function(example):
    """
    Preprocesses a single example using IndicProcessor & Tokenizer.
    """
    src_text = example["translation"][src_lang]
    tgt_text = example["translation"][tgt_lang]

    # Use IndicProcessor for source text preprocessing
    processed_src = ip.preprocess_batch([src_text], src_lang=src_lang, tgt_lang=tgt_lang)[0]

    # Tokenize source text
    model_inputs = en_indic_tokenizer(processed_src, truncation=True, padding="max_length", max_length=128)

    # Tokenize target text (labels for training)
    with en_indic_tokenizer.as_target_tokenizer():
        labels = en_indic_tokenizer(tgt_text, truncation=True, padding="max_length", max_length=128)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing using .map()
tokenized_dataset = dataset.map(preprocess_function)

# ==============================
# Fine-Tuning Step
# ==============================

# Set up fine-tuning parameters
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    per_device_train_batch_size=4,  # Adjust based on GPU memory
    num_train_epochs=3,
    save_steps=500,
    logging_steps=1000,
    evaluation_strategy="no",
    report_to=None,  # Disable logging to external tools
    bf16=True,
    # fp16_full_eval=True
)

# Fine-tune using Hugging Face Trainer
trainer = Trainer(
    model=en_indic_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=en_indic_tokenizer,
)

# en_indic_model.train()
trainer.train()

# Save fine-tuned model & tokenizer
trainer.save_model("./fine_tuned_model")
en_indic_tokenizer.save_pretrained("./fine_tuned_model")

# Set model back to evaluation mode after fine-tuning
en_indic_model.eval()

# ==============================
# Inference (Using Fine-Tuned Model)
# ==============================

# Sample test sentences for inference
en_sents = [
    "When I was young, I used to go to the park every day.",
    "He has many old books, which he inherited from his ancestors.",
    "I can't figure out how to solve my problem.",
    "She is very hardworking and intelligent, which is why she got all the good marks.",
]

# Translate using the fine-tuned model
translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

# Print translations
print(f"\n{src_lang} - {tgt_lang}")
for src, tgt in zip(en_sents, translations):
    print(f"{src_lang}: {src}")
    print(f"{tgt_lang}: {tgt}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/38.6k [00:00<?, ?B/s]

nllb.py:   0%|          | 0.00/9.49k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/5.05M [00:00<?, ?B/s]

nllb_lang_pairs.py:   0%|          | 0.00/81.9k [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bodareddy143 (bodareddy143-bits-pilani) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
